# L’analyse de données et le machine learning avec Python

Le package la plus adaptée pour faire du data mining et du machine learning est la package **scikit-learn**


Ce package propose des fonctions prédéfinies pour un grand nombre de méthodes

- La classification : SVM, plus proches voisins, random forest…
- Les régressions : linéaire, ridge, Lasso…
- Le clustering : k-means…
- L’analyse de données :  ACP, DA…

Dans le cadre de cette formation, l'objectf n'est pas de décrire la théorie des méthodes mais plutôt de comprendre l'utilisation de Python pour les appliquer.

# L’apprentissage supervisé avec Scikit-Learn
Les méthodes d’apprentissage supervisé sont les méthodes actuellement les plus
utilisées en data science. Il s’agit d’essayer de prédire une variable cible et d’utiliser
différentes méthodes pour arriver à cette fin.
Nous allons illustrer ces méthodes de traitement de données avec du code et des
cas pratiques.

### Les données et leur transformation
CPour rappel, il est composé de 3333 individus et de 18 variables. Il est stocké dans un fichier csv, nommé telecom.csv, accessible dans le répertoire Data. On le récupère en utilisant Pandas :

In [2]:
import pandas as pd
import numpy as np

In [3]:
churn=pd.read_csv("./data/telecom.csv")

## Les données

Ce jeu de données n’a pas de données manquantes et nous allons devoir effectuer
quelques transformations pour l’adapter à nos traitements. Nous voyons par exemple qu’il est composé de trois colonnes object.

Nous pouvons afficher les statistiques descriptives pour les colonnes object :

In [5]:
churn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3333 entries, 0 to 3332
Data columns (total 21 columns):
State             3333 non-null object
Account Length    3333 non-null int64
Area Code         3333 non-null int64
Phone             3333 non-null object
Int'l Plan        3333 non-null object
VMail Plan        3333 non-null object
VMail Message     3333 non-null int64
Day Mins          3333 non-null float64
Day Calls         3333 non-null int64
Day Charge        3333 non-null float64
Eve Mins          3333 non-null float64
Eve Calls         3333 non-null int64
Eve Charge        3333 non-null float64
Night Mins        3333 non-null float64
Night Calls       3333 non-null int64
Night Charge      3333 non-null float64
Intl Mins         3333 non-null float64
Intl Calls        3333 non-null int64
Intl Charge       3333 non-null float64
CustServ Calls    3333 non-null int64
Churn?            3333 non-null object
dtypes: float64(8), int64(8), object(5)
memory usage: 546.9+ KB


In [8]:
churn.describe(include="object").transpose()

,count,unique,top,freq
State,3333,51,WV,106
Phone,3333,3333,396-7091,1
Int'l Plan,3333,2,no,3010
VMail Plan,3333,2,no,2411
Churn?,3333,2,False.,2850


## Transformation des données

On voit que les données sont toutes binaires. 

Pour les variables binaires, il nous suffit de les recoder avec Scikit-Learn pour obtenir des données exploitables. Par
ailleurs, il existe une autre variable qualitative dans notre jeu de données, Area Code,
qui est numérique mais avec trois modalités :

In [10]:
churn["Area Code"].value_counts()

415    1655
510     840
408     838
Name: Area Code, dtype: int64

## La préparation des données

Nous allons utiliser le processus de traitement classique pour transformer nos
données avec Scikit-Learn. Dans ce cas, nous n’avons pas de données manquantes,
nous travaillons donc sur la transformation des variables qualitatives.

In [9]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder,Imputer
#from sklearn.impute import SimpleImputer

LabelEncoder va nous permettre de transformer les valeurs textuelles en entiers.
Nous pouvons utiliser pour chaque variable qualitative :

In [10]:
# création d'un objet
encoder=LabelEncoder()
# on applique les données à l'objet
churn["Churn?"]=encoder.fit_transform(churn["Churn?"])

In [12]:
encoder.inverse_transform(churn["Churn?"])

array(['False.', 'False.', 'False.', ..., 'False.', 'False.', 'False.'],
      dtype=object)

In [13]:
encoder.inverse_transform(np.array([0,1]))

array(['False.', 'True.'], dtype=object)

In [15]:
encoder.transform(np.array(["True."]))

array([1], dtype=int64)

In [16]:
dict_label_encode={}
for col in churn.columns:
    if churn[col].dtype == object:
        dict_label_encode[col]=LabelEncoder()
        churn[col]=dict_label_encode[col].fit_transform(churn[col])

In [25]:
dict_label_encode['State'].inverse_transform(churn["State"])

array(['KS', 'OH', 'NJ', ..., 'RI', 'CT', 'TN'], dtype=object)

In [29]:
churn.head()

,State,Account Length,Area Code,Phone,Int'l Plan,VMail Plan,VMail Message,Day Mins,Day Calls,Day Charge,...,Eve Calls,Eve Charge,Night Mins,Night Calls,Night Charge,Intl Mins,Intl Calls,Intl Charge,CustServ Calls,Churn?
0,16,128,415,1926,0,1,25,265.1,110,45.07,...,99,16.78,244.7,91,11.01,10.0,3,2.70,1,0
1,35,107,415,1575,0,1,26,161.6,123,27.47,...,103,16.62,254.4,103,11.45,13.7,3,3.70,1,0
2,31,137,415,1117,0,0,0,243.4,114,41.38,...,110,10.30,162.6,104,7.32,12.2,5,3.29,0,0
3,35,84,408,1707,1,0,0,299.4,71,50.90,...,88,5.26,196.9,89,8.86,6.6,7,1.78,2,0
4,36,75,415,110,1,0,0,166.7,113,28.34,...,122,12.61,186.9,121,8.41,10.1,3,2.73,3,0


## Prédire l’attrition des clients
Lorsqu’on veut prédire une variable binaire, on devra avoir une colonne du type
binaire. On préfère généralement un codage 0/1 afin de garder un type entier simple à gérer. 

Les variables explicatives x auront été préparées de manière intelligente afin de bien appliquer nos modèles.

On crée donc x et y :

In [31]:
x=churn.drop(['State', 'Account Length', 'Area Code', 'Phone',"Churn?"], axis=1)
y=churn["Churn?"]

## Séparation des données

Pour la séparation, on utilise la fonction train_test_split() de Scikit-Learn.

Cette fonction permet de créer automatiquement autant de structures que nécessaire
à partir de nos données. 

Elle utilise une randomisation des individus et ensuite une séparation en fonction d’un paramètre du type test_size :

In [32]:
# on importe la fonction
from sklearn.model_selection import train_test_split
# dans ce cas on a :
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=0)

In [33]:
y.value_counts(normalize=True)

0    0.855086
1    0.144914
Name: Churn?, dtype: float64

In [34]:
y_test.value_counts(normalize=True)

0    0.868066
1    0.131934
Name: Churn?, dtype: float64

Dans certains cas, il peut arriver qu’il y ait une forte disparité de distribution des
modalités entre les proportions d’acceptation et de refus. On peut vouloir faire en
sorte que les répartitions des modalités de y soient égales dans les différents échantillons,
on pourra alors utiliser une stratification. On va utiliser une stratification en
prenant y comme base pour effectuer la stratification :

In [35]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3,stratify = y)
y.value_counts(normalize=True)

0    0.855086
1    0.144914
Name: Churn?, dtype: float64

Ainsi les deux échantillons _train et _test ont la même distribution

In [36]:
y_test.value_counts(normalize=True)

0    0.855
1    0.145
Name: Churn?, dtype: float64

In [37]:
y_train.value_counts(normalize=True)

0    0.855122
1    0.144878
Name: Churn?, dtype: float64

## Le choix et l’ajustement de l’algorithme

Tout au long de ce Notebook, nous allons essayer d'ajouter un nouveau modèle, il s'agit du modèle GBM
```python
from sklearn.ensemble import GradientBoostingClassifier
```

In [116]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression

Ensuite, on crée un objet à partir de la classe du modèle en lui fournissant les
hyperparamètres dont il a besoin :

In [117]:
modele_rf=RandomForestClassifier(n_estimators=100,n_jobs=-1)
modele_knn=KNeighborsClassifier(n_neighbors=5)
modele_gbm = GradientBoostingClassifier()
modele_rn = MLPClassifier()
modele_logit = LogisticRegression()

Dans ce cas, on prend les hyperparamètres par défaut.

On peut ensuite ajuster notre modèle en utilisant les données :

In [118]:
%time modele_rf.fit(x_train,y_train)
%time modele_knn.fit(x_train,y_train)
%time modele_gbm.fit(x_train,y_train)
%time modele_rn.fit(x_train,y_train)
%time modele_logit.fit(x_train,y_train)

pass

Wall time: 253 ms
Wall time: 0 ns
Wall time: 182 ms
Wall time: 525 ms
Wall time: 32.9 ms


C:\Users\s4d-asus-14\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Une fois qu’on a estimé les paramètres du modèle, on va pouvoir extraire des
informations. De nouveaux attributs de chaque classe apparaissent, ils se terminent par le symbole underscore _ :

In [121]:
pd.DataFrame(modele_logit.coef_,columns=x.columns).T

,0
Int'l Plan,2.172762
VMail Plan,-1.057834
VMail Message,0.005937
Day Mins,0.010329
Day Calls,-0.001104
Day Charge,0.004922
Eve Mins,0.006681
Eve Calls,-0.001742
Eve Charge,-0.019875
Night Mins,0.002822


In [57]:
pd.DataFrame(modele_rf.feature_importances_,index=x.columns, 
             columns=["Importances"]).sort_values("Importances",ascending=False)

,Importances
Day Mins,0.138772
Day Charge,0.137367
CustServ Calls,0.105249
Int'l Plan,0.087486
Eve Mins,0.079459
Eve Charge,0.079022
Intl Calls,0.056783
Intl Mins,0.045369
Intl Charge,0.042316
Night Charge,0.038649


In [58]:
pd.DataFrame(modele_gbm.feature_importances_,index=x.columns, 
             columns=["Importances"]).sort_values("Importances",ascending=False)

,Importances
Day Mins,0.167987
CustServ Calls,0.143503
Int'l Plan,0.112142
Day Charge,0.104127
Intl Calls,0.099037
Eve Mins,0.094820
Eve Charge,0.064125
Intl Mins,0.052135
VMail Plan,0.047886
VMail Message,0.045420


Ce qui va nous intéresse avant tout, c’est de prédire avec notre modèle. Pour cela nous allons utiliser la méthode .predict() :

In [122]:
%time y_predict_knn = modele_knn.predict(x_test)
%time y_predict_gbm = modele_gbm.predict(x_test)
%time y_predict_rn = modele_rn.predict(x_test)
%time y_predict_rf = modele_rf.predict(x_test)
%time y_predict_logit = modele_logit.predict(x_test)

Wall time: 29 ms
Wall time: 1.94 ms
Wall time: 2 ms
Wall time: 116 ms
Wall time: 0 ns


On obtient ainsi une valeur prédite pour les éléments de notre échantillon de
validation.

## Les indicateurs pour valider un modèle
La partie validation d’un modèle d’apprentissage supervisé est extrêmement
importante. L’objectif d’un modèle d’apprentissage supervisé est de prédire une
valeur la plus proche possible de la réalité. Nous différencions trois types d’indices
en fonction du type de variable cible. Tous les indicateurs de qualité du modèle sont
stockés dans le module *metrics* de Scikit-Learn.

## Le pourcentage de bien classés
Il s’agit de l’indicateur le plus connu. On le nomme accuracy. Il est calculé à partir du rapport entre le nombre d’individus bien classés et le nombre total d’individus dans l’échantillon.

In [123]:
from sklearn.metrics import accuracy_score, recall_score

accuracy_modele_rf = accuracy_score(y_test,y_predict_rf)
accuracy_modele_knn = accuracy_score(y_test,y_predict_knn)
accuracy_modele_gbm = accuracy_score(y_test,y_predict_gbm)
accuracy_modele_rn = accuracy_score(y_test,y_predict_rn)
accuracy_modele_logit = accuracy_score(y_test,y_predict_logit)


print("Pourcentage de bien classés pour le modèle RF : %.3f" %(accuracy_modele_rf))
print("Pourcentage de bien classés pour le modèle kNN : %.3f" %(accuracy_modele_knn))
print("Pourcentage de bien classés pour le modèle GBM : %.3f" %(accuracy_modele_gbm))
print("Pourcentage de bien classés pour le modèle RN : %.3f" %(accuracy_modele_rn))
print("Pourcentage de bien classés pour le modèle logit : %.3f" %(accuracy_modele_logit))

#recall_modele_rf=recall_score(y_test,y_predict_rf)
#recall_modele_knn=recall_score(y_test,y_predict_knn)
#print("Rappel pour le modèle RF : %.3f" %(recall_modele_rf))
#print("Rappel pour le modèle kNN :%.3f" %(recall_modele_knn))
# ajouter le 3ème modèle

Pourcentage de bien classés pour le modèle RF : 0.958
Pourcentage de bien classés pour le modèle kNN : 0.876
Pourcentage de bien classés pour le modèle GBM : 0.958
Pourcentage de bien classés pour le modèle RN : 0.858
Pourcentage de bien classés pour le modèle logit : 0.850


## La matrice de confusion
Il s’agit d’un autre indicateur important pour juger de la qualité d’un modèle, il n’est pas défini par une seule valeur mais par une matrice dans laquelle on peut lire le croisement entre les valeurs observées et les valeurs prédites à partir du modèle. 

Pour calculer cette matrice, on pourra utiliser :

In [125]:
from sklearn.metrics import confusion_matrix
confusion_matrix_rf=confusion_matrix(y_test,y_predict_rf)
confusion_matrix_knn=confusion_matrix(y_test,y_predict_knn)
confusion_matrix_gbm=confusion_matrix(y_test,y_predict_gbm)
confusion_matrix_rn=confusion_matrix(y_test,y_predict_rn)
confusion_matrix_logit=confusion_matrix(y_test,y_predict_logit)

print("Matrice de confusion pour le modèle RF :",
confusion_matrix_rf, sep="\n")
print("Matrice de confusion pour le modèle kNN :",
confusion_matrix_knn, sep="\n")
print("Matrice de confusion pour le modèle GBM :",
confusion_matrix_gbm, sep="\n")
print("Matrice de confusion pour le modèle RN :",
confusion_matrix_rn, sep="\n")
print("Matrice de confusion pour le modèle Logit :",
confusion_matrix_logit, sep="\n")

# ajouter le 3ème modèle

Matrice de confusion pour le modèle RF :
[[848   7]
 [ 35 110]]
Matrice de confusion pour le modèle kNN :
[[836  19]
 [105  40]]
Matrice de confusion pour le modèle GBM :
[[848   7]
 [ 35 110]]
Matrice de confusion pour le modèle RN :
[[819  36]
 [106  39]]
Matrice de confusion pour le modèle Logit :
[[820  35]
 [115  30]]


## Le rappel (recall), la précision et le f1-score

Scikit-Learn possède des fonctions pour chacun de ces indicateurs, mais il peut
être intéressant d’utiliser une autre fonction qui les affiche pour chaque classe :

tp / (tp + fn)

In [35]:
from sklearn.metrics import classification_report
print("Rapport pour le modèle RF :",
      classification_report(y_test,y_predict_rf) ,sep="\n")

Rapport pour le modèle RF :
              precision    recall  f1-score   support

           0       0.95      0.99      0.97       855
           1       0.95      0.68      0.80       145

   micro avg       0.95      0.95      0.95      1000
   macro avg       0.95      0.84      0.88      1000
weighted avg       0.95      0.95      0.95      1000



In [36]:
print("Rapport pour le modèle kNN :",
      classification_report(y_test,y_predict_knn) ,sep="\n")

Rapport pour le modèle kNN :
              precision    recall  f1-score   support

           0       0.89      0.98      0.93       855
           1       0.66      0.26      0.37       145

   micro avg       0.87      0.87      0.87      1000
   macro avg       0.77      0.62      0.65      1000
weighted avg       0.85      0.87      0.85      1000



In [ ]:
# ajouter le 3ème modèle

## L’aire sous la courbe ROC
La courbe ROC est un indicateur important mais on préfère souvent une valeur plutôt
qu’une courbe afin de comparer nos modèles. Pour cela, on utilise l’aire sous la courbe
ROC (AUC). Cette aire est calculée directement à partir de la courbe ROC. Ainsi, un
modèle aléatoire aura une AUC de 0.5 et un modèle parfait aura une AUC de 1.

In [69]:
from sklearn.metrics import roc_auc_score
auc_modele_rf=roc_auc_score(y_test, modele_rf.predict_proba(x_test)[:,1])
auc_modele_knn=roc_auc_score(y_test,modele_knn.predict_proba(x_test)[:,1])
auc_modele_gbm=roc_auc_score(y_test,modele_gbm.predict_proba(x_test)[:,1])
auc_modele_rn=roc_auc_score(y_test,modele_rn.predict_proba(x_test)[:,1])

print("Aire sous la courbe ROC pour le modèle RF :" ,auc_modele_rf)
print("Aire sous la courbe ROC pour le modèle kNN :" ,auc_modele_knn)
print("Aire sous la courbe ROC pour le modèle GBM :" ,auc_modele_gbm)
print("Aire sous la courbe ROC pour le modèle RN :" ,auc_modele_rn)

# ajouter le 3ème modèle

Aire sous la courbe ROC pour le modèle RF : 0.900262149626941
Aire sous la courbe ROC pour le modèle kNN : 0.6528735632183907
Aire sous la courbe ROC pour le modèle GBM : 0.911058681185723
Aire sous la courbe ROC pour le modèle RN : 0.8154143980641259


## La validation croisée
Jusqu’ici nous avons utilisé des indicateurs basés sur une seule occurrence de test. Ceci veut dire qu’on ne teste notre modèle que sur un seul échantillon.

Une approche alternative souvent utilisée est la validation croisée. Celle-ci est en fait basée sur la répétition de l’estimation et de la validation sur des données différentes.

Pour obtenir ce cv-score, on utilise :

In [39]:
from sklearn.model_selection import cross_val_score

scores_rf = cross_val_score(modele_rf, x_train, y_train, cv=5, scoring='roc_auc')
scores_knn = cross_val_score(modele_knn, x_train, y_train, cv=5, scoring='roc_auc')

print("AUC pour RF : %.3f (+/- %.3f)"% (scores_rf.mean(), scores_rf.std() * 2))
print("AUC pour kNN : %.3f (+/- %.3f)"% (scores_knn.mean(),scores_knn.std() * 2))
# ajouter le 3ème modèle

AUC pour RF : 0.916 (+/- 0.062)
AUC pour kNN : 0.681 (+/- 0.096)


## L’ajustement des hyperparamètres d’un modèle

L’une des tâches du data scientist est de trouver le meilleur modèle possible. La
plupart des modèles de machine learning ont des hyperparamètres. Il s’agit de paramètres
du modèle qui sont définis en amont de l’ajustement.

Scikit-Learn propose une classe GridSearchCV permettant d’implémenter cette
recherche d’hyperparamètres :

In [72]:
from sklearn.model_selection import GridSearchCV

On va donc devoir définir les hyperparamètres que l’on souhaite tester. Pour cela,
on utilisera un dictionnaire d’hyperparamètres, par exemple :

In [87]:
dico_param_rf= {"max_depth":[3,5,7,10,None], "n_estimators":[10,20,50,100,500]}
dico_param_gbm = {"max_depth":[3,5,7,10], "n_estimators":[10,20,50,100,500]}


On va encore utiliser l’accuracy pour valider notre modèle. Finalement, nous allons
utiliser une validation croisée à cinq groupes pour valider les résultats.
Le nouvel objet est le suivant :

In [88]:
recherche_hyper_rf = GridSearchCV(RandomForestClassifier(), 
                               dico_param_rf, 
                               scoring="roc_auc",cv=5)

In [89]:
recherche_hyper_gbm = GridSearchCV(GradientBoostingClassifier(), 
                               dico_param_gbm, 
                               scoring="roc_auc",cv=5)

Une fois qu’on a créé cet objet, on peut lui joindre les données afin d’estimer les
meilleurs paramètres du modèle.

Cette étape peut être très longue.

In [90]:
recherche_hyper_rf.fit(x_train, y_train)
pass

In [91]:
recherche_hyper_gbm.fit(x_train,y_train)
pass

In [92]:
print("Meilleurs paramètres pour RF :", recherche_hyper_rf.best_params_)
print("Meilleurs paramètres pour GBM :", recherche_hyper_gbm.best_params_)

Meilleurs paramètres pour RF : {'max_depth': None, 'n_estimators': 100}
Meilleurs paramètres pour GBM : {'max_depth': 3, 'n_estimators': 100}


In [93]:
print("AUC RF :",recherche_hyper_rf.best_score_)
print("AUC GBM :",recherche_hyper_gbm.best_score_)

AUC RF : 0.9169238672727991
AUC GBM : 0.9188367026926222


In [94]:
accuracy_score(y_test,recherche_hyper_gbm.predict(x_test))

0.958

In [95]:
accuracy_score(y_test,recherche_hyper_rf.predict(x_test))

0.951

In [102]:
pd.DataFrame(recherche_hyper_gbm.cv_results_)[["params",
                                               "mean_test_score",
                                               "std_test_score"]].sort_values("mean_test_score",
                                                                             ascending=False)

,params,mean_test_score,std_test_score
3,"{'max_depth': 3, 'n_estimators': 100}",0.918837,0.030249
7,"{'max_depth': 5, 'n_estimators': 50}",0.915535,0.032923
8,"{'max_depth': 5, 'n_estimators': 100}",0.913577,0.035799
2,"{'max_depth': 3, 'n_estimators': 50}",0.913434,0.033358
6,"{'max_depth': 5, 'n_estimators': 20}",0.910473,0.032149
4,"{'max_depth': 3, 'n_estimators': 500}",0.909525,0.034806
5,"{'max_depth': 5, 'n_estimators': 10}",0.907062,0.033641
19,"{'max_depth': 10, 'n_estimators': 500}",0.906464,0.040196
18,"{'max_depth': 10, 'n_estimators': 100}",0.906252,0.036200
14,"{'max_depth': 7, 'n_estimators': 500}",0.905474,0.040859


In [108]:
recherche_hyper_gbm.predict_proba(x_test)[:,1]

array([0.04243261, 0.0297343 , 0.02579451, 0.01480455, 0.04962169,
       0.03277675, 0.02971974, 0.89536764, 0.02768316, 0.09767683,
       0.01098248, 0.01159032, 0.02078861, 0.09486473, 0.02950235,
       0.01934761, 0.07057012, 0.02358438, 0.09953591, 0.08059043,
       0.25790503, 0.03065299, 0.82664506, 0.02969217, 0.01950334,
       0.7519109 , 0.03813113, 0.01840815, 0.05476683, 0.02165875,
       0.01781736, 0.02826266, 0.06064501, 0.21887895, 0.98270043,
       0.02312148, 0.03145732, 0.01098071, 0.0212649 , 0.02946342,
       0.65838014, 0.01813464, 0.03781701, 0.94896615, 0.02133946,
       0.02331114, 0.03461429, 0.01742984, 0.02999226, 0.03332499,
       0.02799447, 0.02664821, 0.02355094, 0.03615799, 0.01770138,
       0.02361751, 0.03224086, 0.06373301, 0.02316135, 0.18623757,
       0.4949015 , 0.0109937 , 0.01444412, 0.03823243, 0.03275986,
       0.03658721, 0.01534016, 0.02053192, 0.02695769, 0.05955305,
       0.03366808, 0.02172151, 0.02114332, 0.01978235, 0.07431

**Exercice :**

Effectuez le même travail avc GBM

## La construction d’un pipeline de traitement

Bien souvent vous allez être amené à enchaîner des traitements sur des données.
On peut bien sûr développer son code de manière à suivre les étapes une à une mais il est souvent plus intéressant de créer des suites de traitements automatisées avec Scikit-Learn. 

Ces suites de traitements sont appelées pipeline. Ils simplifieront votre code et permettront de passer en production simplement.

Ainsi, on va pouvoir faire une analyse en composantes principales suivies d’un
algorithme de plus proches voisins directement dans un pipeline :

In [110]:
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.decomposition import PCA

In [111]:
acp=PCA(n_components=8)
knn=KNeighborsClassifier()
rf=RandomForestClassifier()
pipe=Pipeline(steps=[("acp",acp),("knn",knn)])
pipe2=Pipeline(steps=[("acp",acp),("rf",rf)])

pipe.fit(x_train, y_train)
pipe2.fit(x_train, y_train)
pass

C:\Users\s4d-asus-14\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


On a ainsi enchaîné deux traitements. Si on cherche des sorties liées à chacune
des étapes, on pourra le faire simplement. Par exemple, si l’objectif est d’extraire la part de variances expliquées par les composantes de l’analyse en composantes principales, on fera :

In [113]:
accuracy_score(y_test,pipe.predict(x_test))

0.875

In [58]:
pipe.named_steps["acp"].explained_variance_ratio_

array([0.31289476, 0.28288035, 0.26311168, 0.04080837, 0.03990796,
       0.0391955 , 0.01951312, 0.00085234])

## Trouver la meilleure combinaison d’hyperparamètres dans un pipeline

Essayons de trouver la meilleure combinaison d’hyperparamètres dans un pipeline.
Dans le cadre de cet exemple, nous utiliserons les SVM (support vector machines,
également appelés séparateurs à vaste marge ou machines à vecteurs de support).
Ce sont des méthodes assez complexes dans leur principe mais simples dans leur
mise en oeuvre.

Les hyperparamètres d’un modèle de SVM sont assez nombreux. Les plus importants
étant le noyau choisi (linéaire, polynomial, sigmoïd, RBF…), les paramètres de
ces noyaux (le degré pour le cas polynomiale, gamma…) et le C pour la marge floue.

Dans notre exemple, on utilisera la fonction make_pipeline qui est équivalente
à la classe précédente avec quelques simplifications :

In [37]:
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV

# construction du pipeline basé sur deux approches
mon_pipe=make_pipeline(PCA(), SVC(probability=True))

# construction du dictionnaire des paramètres
# (attention utilisation de __)
param_grid = dict(pca__n_components=[5, 10, x_train.shape[1]],
                  svc__C= [1, 10, 100, 1000],
                  svc__kernel= ['sigmoid', 'rbf'],
                  svc__gamma= [0.001, 0.0001])

# on construit l’objet GridSearch et on estime les hyper-paramètres
# par validation croisée
grid_search_mon_pipe = GridSearchCV(mon_pipe, param_grid = param_grid, scoring = "roc_auc", cv = 4)

In [38]:
grid_search_mon_pipe.fit(x_train,y_train)
pass

In [39]:
# la meilleure combinaisons de paramètres est :
grid_search_mon_pipe.best_params_

{'pca__n_components': 16,
 'svc__C': 10,
 'svc__gamma': 0.0001,
 'svc__kernel': 'rbf'}

In [40]:
grid_search_mon_pipe.best_score_

0.7702776709885911

Les meilleurs hyperparamètres obtenus en utilisant l’aire sous la courbe ROC sont
la combinaison C = 10, gamma = 0.0001, un noyau RBF pour les SVM et dix composantes
pour notre analyse en composantes principales.

Dans ce code, on définit les hyperparamètres associés à une méthode du pipeline
avec un double underscore : __.

L’utilisation des pipelines de Scikit-Learn va devenir rapidement une étape cruciale
de vos développements en Python.

## Passer en production votre modèle d’apprentissage supervisé

### Persistance de modèle avec Scikit-Learn

Python possède plusieurs outils pour la persistance d’objets, c’est-à-dire pour stocker
des objets dans des fichiers. Les objets de Scikit-Learn sont aussi dans cette
situation. On utilise un format pickle qui aura l’extension .pkl.

Par exemple, si nous voulons sauvegarder le dernier pipeline de traitement, nous
allons utiliser :

In [115]:
from sklearn.externals import joblib
joblib.dump(recherche_hyper_gbm, './data/hyper_gbm.pkl')

['./data/hyper_gbm.pkl']

Une fois ce modèle stocké, on peut très bien le réutiliser dans un autre cadre. Si
nous créons un nouveau notebook, nous allons utiliser :



In [65]:
from sklearn.externals import joblib
grid_search_mon_pipe = joblib.load('../data/modele_grid_pipe.pkl')

On peut ensuite appliquer le modèle avec tous les paramètres qui ont été appris :


```python
grid_search_mon_pipe.predict(x_test)
```

L’utilisation d’un fichier Pickle dans un notebook est une technique assez simple et courante.